In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoHumanNontataPromoters
from utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config
from cnn_model import CNN

## Config

In [37]:
config = {
    "use_padding": False,
    "run_on_gpu": True,
    "dataset": DemoHumanNontataPromoters,
    "number_of_classes": 1,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
#   vocabulary that is not present in the training set but is present in the test set
    "vocab_to_add": ["N"],
}
check_config(config)

config is correct


## Choose the dataset

In [38]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/24 [00:00<?, ?it/s]

## Tokenizer and vocab

In [5]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])
if(config["vocab_to_add"]):
    for token in config["vocab_to_add"]:
        vocabulary.append_token(token)

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'N': 7, '<pad>': 6, 'T': 3, '<eos>': 5, 'G': 2, 'C': 4, 'A': 1, '<bos>': 0}


## Dataloader and batch preparation

In [10]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  251


## Model

In [11]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [12]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/home/jupyter/genomic_benchmarks/notebooks/utils.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 81.6%, Avg loss: 0.615950 

Epoch 1
Train metrics: 
 Accuracy: 81.3%, Avg loss: 0.613995 

Epoch 2
Train metrics: 
 Accuracy: 82.4%, Avg loss: 0.602526 

Epoch 3
Train metrics: 
 Accuracy: 81.1%, Avg loss: 0.603079 

Epoch 4
Train metrics: 
 Accuracy: 83.9%, Avg loss: 0.596460 

Epoch 5
Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.594147 

Epoch 6
Train metrics: 
 Accuracy: 83.0%, Avg loss: 0.600087 

Epoch 7
Train metrics: 
 Accuracy: 84.0%, Avg loss: 0.592090 

Epoch 8
Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.592180 

Epoch 9
Train metrics: 
 Accuracy: 84.8%, Avg loss: 0.590149 

Epoch 10
Train metrics: 
 Accuracy: 85.5%, Avg loss: 0.589941 

Epoch 11
Train metrics: 
 Accuracy: 85.5%, Avg loss: 0.587873 

Epoch 12
Train metrics: 
 Accuracy: 85.0%, Avg loss: 0.588353 

Epoch 13
Train metrics: 
 Accuracy: 86.7%, Avg loss: 0.588085 

Epoch 14
Train metrics: 
 Accuracy: 86.4%, Avg loss: 0.585746 



## Testing

In [13]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/24 [00:00<?, ?it/s]

test_loss  169.54310023784637
num_batches 283
correct 7557
size 9034
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.599092 

